<a href="https://colab.research.google.com/github/BWetende/Bilha.Wetende1/blob/main/Recommender_System_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: Bilha Ann Wetende

Registration Number: SDS6/43805/2023

Course Code: SMA 6114

Course Name: Matrix Algebra Computations

Project Title: Movies Recommender System

Data Source: grouplens.org/datasets/movielens/ml-1m.zip

Introduction

A recommendation system/recommender system is a system that uses Machine Learning to predict or filter a user's preferences based on their prior decisions and actions.

In this project,Singular Value Decomposition technique is used in filtering to make personalised recommendations based on previous ratings and the naming of the movies.

The steps employed include:
i. Data Preparation where we gather and manipulate the data in preparation for use in the project. It includes removal of duplicates, importing of data, and ensuring accuracy in the data being used.

ii. Employing Singular Value Decomposition where the user matrix is decomposed into  U, sigma and V

iii. Dimensionality Reduction where the user item matrix which is decomposed using SVD is truncated by maintaining only the top K singular values and their corresponding columns in U and V. This step reduces the dimensionality of the original matrix while maintaining its key features to ensure accuract.

iv. Making Recommendations where the user matrix is multiplied by the truncated sigma matrix and then multiplied by the transposed item vectors. This product is the recommendation score where higher score indicate more preference.

v. Top-N Recommendations where the top N items are presented to the user through the user interface where the scores are sorted in a descending order

vi. Evaluation & Iteration where the accuracy of the recommendation system is evaluated using evaluation metrics or for our case, feedback from further analysis of the recommendation result by end users

In this project, we build a movies recommendation system like the one used for netflix and youtube to give users a recommendation of movies that are of similar naming and genre and with a higher rating from the movies they search.

To develop this model we first create a search engine that searches for the movies by names, then develop a score from the general ratings and individual user ratings; which will be used to sort the likelihood of a movie being similar to that in our search

Step 1a: The first step is to mount google drive into google collab in order to read the dataset in Python

In [ ]:
# reading google drive into google collab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 1b: The next step is to load the data into python using the google collab software and from our link in google drive. Note that the type of dataset is separated by double colons with no header and the specified column names under names. The dataset is encoded in latin-1

In [ ]:
#importing the dataset and necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
movies = pd.read_csv('//content/drive/MyDrive/movies.dat', sep='::', header=None, names=['movie_id', 'title', 'genres'], encoding='latin-1')
ratings = pd.read_csv('/content/drive/MyDrive/ratings.dat', sep='::', engine='python', header=None, names=['userId', 'movieId', 'rating', 'timestamp'], encoding='latin-1')
users = pd.read_csv('/content/drive/MyDrive/users.dat', sep='::', engine='python', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], encoding='latin-1')

<ipython-input-2-b025bcf94501>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('//content/drive/MyDrive/movies.dat', sep='::', header=None, names=['movie_id', 'title', 'genres'], encoding='latin-1')


Here is a snipet of the movies dataset. Please note that the clean_title column has been added in the course of the project. It contains the information on the movie names and genres. Its primary key is the movie_id

In [ ]:
movies

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama


Here is a snipet of the users dataset. This dataset shows us the information of the users who are viewers of the movies. Its primary key is the UserID

In [ ]:
users

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


Below is a snipet of the ratings dataset which contains the ratings for the specific movies. Its prin=mary key is the MovieId

In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


Step 2a: Cleaning up the movies dataset

To develop the search engine which will search the movie names by names, we first clean up the title column and come up with the clean_title which will be used for the search.

We remove the brackets from the movie names as we need to search the movie names in pairs and the bracket is unneccesary. This is done using the re.sub function.

We then use the apply function to create a column in the movies dataset that contains the clean_title, which will be used for our search.

In [ ]:
#Cleaning the movie titles with regex (regular expression)

import re
#movies['year'] = movies['title'].str.extract('\((\d{4})\)$', expand=True)
#movies['title'] = movies['title'].str.replace('\s*\(\d{4}\)$', '')

def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "", title)

#creating a new column called clean_title
movies["clean_title"] = movies["title"].apply(clean_title)
movies

,movie_id,title,genres,clean_title
0,1,Toy Story (1995),Animation|Children's|Comedy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,Meet the Parents 2000
3879,3949,Requiem for a Dream (2000),Drama,Requiem for a Dream 2000
3880,3950,Tigerland (2000),Drama,Tigerland 2000
3881,3951,Two Family House (2000),Drama,Two Family House 2000


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


Step 2b: Building a search engine

We use the TfidfVectorizer to build a search engine that will search for the movies with similar names in pairs i.e. for a movie called "Toy Story 1995" the search engine will search for movies with names similar to "Toy Story" and "Story 1995" and give us the movie titles that are most similar from the clean_title column

In [ ]:
#Building a search engine
from sklearn.feature_extraction.text import TfidfVectorizer

#creating a paramenter that can search two multiple words ie Toy Story & Story 1995
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

Step 2c: We define the function Query which will be used to define the movie name under search.

We also specify the number of results needed i.e the top 10 movies in similarity to the one under search

In [ ]:
#creating a search function ie creating a similarity between a term that we search and the movies using cosine similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Create a TfidfVectorizer to convert movie titles into vectors
vectorizer = TfidfVectorizer()

# Convert the list of movie titles to a matrix of TF-IDF features
tfidf = vectorizer.fit_transform(movies["title"].apply(clean_title))

# Create a function to search for similar movies
def search(query):
    # Clean the query string
    query = clean_title(query)

    # Convert the query string to a vector using the same vectorizer
    query_vec = vectorizer.transform([query])

    # Compute the cosine similarity between the query vector and all movie vectors
    similarity = cosine_similarity(query_vec, tfidf).flatten()

    # Sort the movies by similarity score
    sim_scores = list(enumerate(similarity))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Return the top 10 most similar movies
    top_similar_movies = [(movies.iloc[i]["title"], score) for i, score in sim_scores[:10]]
    return top_similar_movies

Step 2d: We then illustrate the search function Query with the movie titled "My Family" to test if it works

In [ ]:
# Define a query string
query = "My Family" #Illustration, could be any movie title

# Search for similar movies
results = search(query)

# Print the results
print(f"Top 10 movies similar to '{query}':")
for title, score in results:
    print(f"\t{title} ({score:.2f})")

Top 10 movies similar to 'My Family':
	My Family (1995) (0.93)
	Family Thing, A (1996) (0.53)
	Perez Family, The (1995) (0.47)
	Two Family House (2000) (0.46)
	Addams Family, The (1991) (0.45)
	Family Plot (1976) (0.44)
	Addams Family Values (1993) (0.38)
	My Life (1993) (0.37)
	Swiss Family Robinson (1960) (0.36)
	Stuart Saves His Family (1995) (0.34)


Step 3: Building a Recommendation System

Here, we develop a recommendation system from the ratings & users dataset to further include two parameters in our search engine: the overall rating from all users (the general population including people who have not viewed the movies) which is from the ratings dataset; and the ratings from users who have viewed the movies and like movies of similar genres from the users dataset

We view the Ratings dataset

In [ ]:
#Creating a recommendation system
#reading the ratings dataset
ratings = pd.read_csv('/content/drive/MyDrive/ratings.dat', sep='::', engine='python', header=None, names=['userId', 'movieId', 'rating', 'timestamp'], encoding='latin-1')
ratings


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


We view the ratings datatypes which is int

In [ ]:
#viewing the datatype for the ratings dataset
ratings.dtypes

userId       int64
movieId      int64
rating       int64
timestamp    int64
dtype: object

Here we filter movies with a rating > 4. We use the unique() function to remove duplicates.

We develop the function similar_users which helps us to find the users who like the movies which have also been rated > 4. This gives us an array of the movieIds that fall under that category

We illustrate using movieId 279

In [ ]:
#the movie id for "My Family" is 279
#people who liked similar movie
movie_id = 279 #Illustration
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] >= 4)]["userId"].unique()
similar_users

array([  62,  150,  187,  293,  319,  352,  469,  524,  752,  801, 1448,
       1605, 1701, 1758, 1980, 2230, 2238, 2822, 3163, 3246, 3271, 3519,
       3729, 4082, 4169, 4277, 4330, 4340, 4387, 4749, 4782, 4808])

We also filter the movies with a rating > 4 from the users dataset to find similar_user_recs

In [ ]:
# other movies that were liked by people who liked "My Family"
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
similar_user_recs

9015      3789
9022      1254
9024      1256
9036      3871
9050      2069
          ... 
803146    2019
803149     541
803157    3763
803168    3770
803177    2046
Name: movieId, Length: 3646, dtype: int64

We then find a percentage of the counts of similar_users_rec out of the total count of unique similar_users

In [ ]:
#counts how many times each movie appears in the dataset divided by the total number of users to find %
#similar_user_recs.value_counts() / len (similar_users)
movie_counts = similar_user_recs.value_counts()
movie_percents = movie_counts / len(similar_users)
movie_percents

2858    0.56250
593     0.53125
318     0.53125
527     0.50000
1193    0.50000
         ...   
2426    0.03125
2276    0.03125
3869    0.03125
370     0.03125
2046    0.03125
Name: movieId, Length: 1358, dtype: float64

We then filter out to see movies that > 10% of similar users liked out of the above percentage

In [ ]:
# movies that only > 10% of the similar users liked
recommended_movies = movie_percents[movie_percents > 0.1]
recommended_movies

2858    0.56250
593     0.53125
318     0.53125
527     0.50000
1193    0.50000
         ...   
589     0.12500
1535    0.12500
1270    0.12500
1271    0.12500
562     0.12500
Name: movieId, Length: 326, dtype: float64

We then tabulate movies that are most liked out of the > 10% population of similar users; with ratings > 4

In [ ]:
# Finding movies that most people like that are also similar to what others like most
all_users = ratings[(ratings["movieId"].isin(recommended_movies.index)) & (ratings["rating"] > 4)]
all_users

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
4,1,2355,5,978824291
7,1,2804,5,978300719
10,1,595,5,978824268
14,1,1035,5,978301753
...,...,...,...,...
1000186,6040,527,5,956704219
1000192,6040,2019,5,956703977
1000200,6040,2028,5,956704519
1000205,6040,1094,5,956704887


We the find the percentage of all users (not just similar users who are also viewers) who liked the same movies that were recommended by similar users

In [ ]:
# Percentage of all users that recommend each of the recommended movies
all_users_count = all_users["movieId"].value_counts()
all_users_percents = all_users_count / len(all_users["userId"].unique())
all_users_percents

2858    0.329197
260     0.306222
1198    0.251551
1196    0.248700
858     0.247359
          ...   
1611    0.006037
82      0.005366
627     0.004025
1535    0.002683
279     0.001342
Name: movieId, Length: 326, dtype: float64

We use the pandas concat method to combine the two percentages into one table

In [ ]:
# using pandas concat method to compare the two
rec_percentages = pd.concat([recommended_movies, all_users_percents], axis=1)
rec_percentages.columns = ["similar", "all"]
rec_percentages


,similar,all
2858,0.56250,0.329197
593,0.53125,0.226396
318,0.53125,0.244340
527,0.50000,0.247359
1193,0.50000,0.157136
...,...,...
589,0.12500,0.157974
1535,0.12500,0.002683
1270,0.12500,0.132651
1271,0.12500,0.025323


We calculate the score which will be the index for the recommendation by calculating the similar user percentage of movies with ratings > 4 vs the all user percentage of movies with ratings > 4

In [ ]:
# creating a recommendation score
rec_percentages["score"] = rec_percentages["similar"]/rec_percentages["all"]

#using the pandas sort method to sort from the top recommendation fo that we can filter out the most recommended
rec_percentages = rec_percentages.sort_values("score", ascending = False)
rec_percentages

,similar,all,score
279,0.25000,0.001342,186.343750
1535,0.12500,0.002683,46.585937
82,0.18750,0.005366,34.939453
1810,0.21875,0.006373,34.326480
627,0.12500,0.004025,31.057292
...,...,...,...
260,0.31250,0.306222,1.020502
1196,0.25000,0.248700,1.005226
1200,0.12500,0.126446,0.988561
1270,0.12500,0.132651,0.942320


We sort the score in descending order to give us the top values in similarity; and we condition the results to 10 recommendations.

We then comine the tables to give us the output of Movieid, title, genres, similar, all and the score

In [ ]:
# taking the top 10 recommendation and merging with the movies dataset so that it can give us the movie names not just ids
top_recommendations = rec_percentages.sort_values(by='similar', ascending=False).head(10).reset_index()
top_movies = pd.merge(top_recommendations, movies, how='left', left_on='index', right_on='movie_id')
top_movies = top_movies[['movie_id', 'title','genres','similar', 'all','score']]
top_movies


,movie_id,title,genres,similar,all,score
0,2858,American Beauty (1999),Comedy|Drama,0.56250,0.329197,1.708705
1,593,"Silence of the Lambs, The (1991)",Drama|Thriller,0.53125,0.226396,2.346551
2,318,"Shawshank Redemption, The (1994)",Drama,0.53125,0.244340,2.174224
3,527,Schindler's List (1993),Drama|War,0.50000,0.247359,2.021356
4,1193,One Flew Over the Cuckoo's Nest (1975),Drama,0.50000,0.157136,3.181964
5,2762,"Sixth Sense, The (1999)",Thriller,0.40625,0.232266,1.749075
6,2908,Boys Don't Cry (1999),Drama,0.40625,0.055006,7.385575
7,608,Fargo (1996),Crime|Drama|Thriller,0.40625,0.214322,1.895515
8,296,Pulp Fiction (1994),Crime|Drama,0.40625,0.200067,2.030569
9,1358,Sling Blade (1996),Drama|Thriller,0.40625,0.084689,4.796968


We the develop an interractive search tab using the ipywidgets package that will be used as a search tab for the movie entries in order to give us the recommendations

In [ ]:
#Creating a search tab

import ipywidgets as widgets
from IPython.display import display

# Define the search bar widget
search_bar = widgets.Text(
    value="",
    placeholder="Enter a movie title",
    description="Movie Title:",
    disabled=False
)

# Define the search results widget
search_results = widgets.Output()

# Define the search function
def on_search(btn):
    # Clear the previous search results
    search_results.clear_output()

    # Perform the search
    results = search(search_bar.value)

    # Display the search results
    with search_results:
        for result in results:
            print(result[0], " (score:", result[1], ")")

# Define the search button widget
search_btn = widgets.Button(
    description="Search",
    disabled=False,
    button_style="",
    tooltip="Search for movies"
)

# Attach the search function to the search button
search_btn.on_click(on_search)

# Display the search bar, search button, and search results widgets
display(widgets.HBox([search_bar, search_btn]))
display(search_results)

Output()